<a href="https://colab.research.google.com/github/simbakm/training_yolov11_notebook/blob/main/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **starting Point @ simbakm Training YOlo  version 11 bold text**

# Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://clickpy.clickhouse.com/dashboard/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [11]:
!uv pip install ultralytics
!pip install roboflow kagglehub
from collections import defaultdict
from ultralytics import YOLO
from google.colab import files
import ultralytics
from glob import glob
from tqdm import tqdm
import os, random, shutil
import json
ultralytics.checks()



Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 46.8/112.6 GB disk)


**check for GPU**

In [3]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")

Using cuda device.


**STEP 2 — Download the Roboflow dataset (YOLOv11-ready)bold text**

**helper fuction to couint the number of files**

In [4]:
import os
from collections import defaultdict

def count_file_types(dataset_path):
    file_counts = defaultdict(int)

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # get file extension
            file_counts[ext] += 1

    print("File type counts:\n")
    for ext, count in sorted(file_counts.items()):
        print(f"{ext if ext else 'No Extension'} : {count}")

    return dict(file_counts)


In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="YdrZFjwWblcZBkXrqqaR")
project = rf.workspace("stairs-ttqqp").project("trash-bin-asn0s")
dataset = project.version(1).download("yolov11")

roboflow_path = dataset.location
print("Roboflow dataset:", roboflow_path)

counts = count_file_types(roboflow_path)



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to trash-bin-1 in yolov11:: 100%|██████████| 902/902 [00:00<00:00, 8213.15it/s]

Roboflow dataset: /content/trash-bin-1
File type counts:

.jpg : 445
.txt : 447
.yaml : 1


## **STEP 3: Download the TACO dataset from KaggleHub**

In [6]:
import kagglehub
path = kagglehub.dataset_download("kneroma/tacotrashdataset")
print("Downloaded to:", path)
counts = count_file_types(path)


Using Colab cache for faster access to the 'tacotrashdataset' dataset.
Downloaded to: /kaggle/input/tacotrashdataset
File type counts:

.bin : 1
.csv : 1
.jpg : 1500
.json : 1
.txt : 1


**STEP 4 — Convert TACO (COCO JSON) → YOLO format**

In [9]:


# TACO dataset path (from KaggleHub)
taco_path = path
data_dir = os.path.join(taco_path, "data")

# Load annotations
annotations_path = os.path.join(data_dir, "annotations.json")
annotations = json.load(open(annotations_path))

# Output
output_taco = "/content/taco_yolo"
#delete privous derectories
if os.path.exists(output_taco):
    shutil.rmtree(output_taco)
    print("Old taco_yolo directory removed.")
else:
    print("No previous taco_yolo folder found.")

os.makedirs(f"{output_taco}/images", exist_ok=True)
os.makedirs(f"{output_taco}/labels", exist_ok=True)

# Map image id → image info
image_map = {img["id"]: img for img in annotations["images"]}

# Find all images recursively
all_images = {}
image_files = glob(f"{data_dir}/**/*.jpg", recursive=True) + \
              glob(f"{data_dir}/**/*.JPG", recursive=True)

for f in image_files:
    all_images[os.path.relpath(f, data_dir)] = f


print("Total images found:", len(all_images))

# Group annotations by image ID
ann_by_image = {}
for ann in annotations["annotations"]:
    img_id = ann["image_id"]
    ann_by_image.setdefault(img_id, []).append(ann)

# Convert to YOLO
for img_id, img_info in tqdm(image_map.items()):
    filename = img_info["file_name"]
    batch_relative_path = filename  # contains batch_x/000000.jpg

    if batch_relative_path not in all_images:
        print("Missing image:", batch_relative_path)
        continue

    src = all_images[batch_relative_path]

    # create unique new filename based on img_id
    new_name = f"{img_id}.jpg"
    dst = f"{output_taco}/images/{new_name}"

    shutil.copy(src, dst)

    # image width/height
    W = img_info["width"]
    H = img_info["height"]

    # label path (same name as image)
    label_path = f"{output_taco}/labels/{img_id}.txt"
    with open(label_path, "w") as f:
        for ann in ann_by_image.get(img_id, []):
            x, y, w, h = ann["bbox"]

            # convert to YOLO
            xc = (x + w/2) / W
            yc = (y + h/2) / H
            w /= W
            h /= H
            # Create category id mapping
            categories = annotations["categories"]
            cat_id_map = {cat["id"]: i for i, cat in enumerate(categories)}
            class_id = cat_id_map[ann["category_id"]]

            f.write(f"{class_id} {xc} {yc} {w} {h}\n")

print("DONE — TACO converted to YOLO:", output_taco)


Old taco_yolo directory removed.
Total images found: 1500


100%|██████████| 1500/1500 [00:33<00:00, 44.53it/s]

DONE — TACO converted to YOLO: /content/taco_yolo


# **5: Split the dataset into Train,val and test using ratio 80|10|10**

In [10]:


# Clean up old splits first
if os.path.exists("/content/taco_split"):
    print("Removing old splits...")
    shutil.rmtree("/content/taco_split")

taco_img = "/content/taco_yolo/images"
taco_lbl = "/content/taco_yolo/labels"

# Verify source files exist
images = sorted(glob(f"{taco_img}/*.jpg"))
if len(images) == 0:
    print("❌ No images found! Check paths.")
else:
    print(f"✅ Found {len(images)} images")
    random.shuffle(images)

    n = len(images)
    train_split = int(n * 0.8)
    val_split = int(n * 0.9)

    train_files = images[:train_split]
    val_files   = images[train_split:val_split]
    test_files  = images[val_split:]

    def copy_split(files, folder):
        os.makedirs(f"/content/taco_split/images/{folder}", exist_ok=True)
        os.makedirs(f"/content/taco_split/labels/{folder}", exist_ok=True)
        for img in files:
            lbl = img.replace("images", "labels").replace(".jpg", ".txt")
            if os.path.exists(lbl):
                shutil.copy(img, f"/content/taco_split/images/{folder}/")
                shutil.copy(lbl, f"/content/taco_split/labels/{folder}/")
            else:
                print(f"⚠️  Warning: No label for {img}")

    copy_split(train_files, "train")
    copy_split(val_files, "val")
    copy_split(test_files, "test")

    print(f"\nTACO split complete!")
    print(f"  Train: {len(train_files)} images")
    print(f"  Val: {len(val_files)} images")
    print(f"  Test: {len(test_files)} images")

✅ Found 1500 images

TACO split complete!
  Train: 1200 images
  Val: 150 images
  Test: 150 images


**Group TACO Classes into 5 & Shift IDs**
# !!!! Do not run twice this cell !!!!

In [13]:


# Correct TACO class mapping based on actual class IDs
# Source: https://github.com/pedropro/TACO/blob/master/scripts/class_mapping.txt

taco_group_map_5 = {
    # Class 0: Plastic Items (ALL plastic-related classes)
    0:0,   # plastic_film
    1:0,   # other_plastic
    2:0,   # plastic_bag
    3:0,   # plastic_bottle_cap
    4:0,   # plastic_bottle
    5:0,   # clear_plastic_bottle
    6:0,   # plastic_cup_lid
    7:0,   # plastic_cup
    8:0,   # disposable_plastic_cup
    9:0,   # plastic_container
    10:0,  # plastic_jar
    11:0,  # plastic_lid
    12:0,  # plastic_plate
    13:0,  # plastic_knife
    14:0,  # plastic_fork
    15:0,  # plastic_spoon
    16:0,  # plastic_scoop
    17:0,  # plastic_straw
    18:0,  # plastic_other
    55:0,  # styrofoam_piece
    56:0,  # styrofoam_cup
    57:0,  # food_container
    58:0,  # other_plastic_wrapper
    59:0,  # plastic_film_wrapper

    # Class 1: Metal Items
    19:1,  # drink_can
    20:1,  # metal_bottle_cap
    21:1,  # aluminium_foil
    22:1,  # pop_tab
    23:1,  # metal_lid
    24:1,  # metal_can
    25:1,  # metal_jar

    # Class 2: Paper & Cardboard
    26:2,  # normal_paper
    27:2,  # paper_cup
    28:2,  # corrugated_carton
    29:2,  # drink_carton
    30:2,  # pizza_box
    31:2,  # paper_bag
    32:2,  # cardboard_box
    33:2,  # egg_carton
    34:2,  # paper_tray
    35:2,  # paper_plate
    36:2,  # cardboard_packaging
    37:2,  # tissue_paper
    38:2,  # napkin
    39:2,  # paper_wrapper

    # Class 3: Glass Items
    40:3,  # glass_bottle
    41:3,  # glass_jar
    42:3,  # broken_glass

    # Class 4: Other & Unlabeled Litter
    43:4,  # cigarette
    44:4,  # unlabeled_litter
    45:4,  # food_waste
    46:4,  # organic
    47:4,  # battery
    48:4,  # electronic_waste
    49:4,  # clothing
    50:4,  # shoe
    51:4,  # textile
    52:4,  # rubber_band
    53:4,  # tire
    54:4,  # rubber_other
}

# Dictionary to count ANNOTATIONS per class
annotation_counts = defaultdict(int)
image_counts = defaultdict(int)
total_images_processed = set()

# Process each split
for split in ["train", "val", "test"]:
    lbl_files = glob(f"/content/taco_split/labels/{split}/*.txt")
    for file in lbl_files:
        new_lines = []
        classes_in_image = set()

        with open(file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) > 0:
                    original_id = int(parts[0])
                    if original_id in taco_group_map_5:
                        grouped_id = taco_group_map_5[original_id]
                        parts[0] = str(grouped_id)
                        new_lines.append(" ".join(parts))

                        # Count this annotation
                        annotation_counts[grouped_id] += 1
                        classes_in_image.add(grouped_id)

        # Write the updated labels back to file
        with open(file, "w") as f:
            f.write("\n".join(new_lines))

        # Count images per class
        for class_id in classes_in_image:
            image_counts[class_id] += 1

        total_images_processed.add(file)

# Print results
print("\n" + "="*60)
print("TACO GROUPED INTO 5 CLASSES - CORRECTED MAPPING")
print("="*60)

class_names_5 = {
    0: "Plastic Items",
    1: "Metal Items",
    2: "Paper & Cardboard",
    3: "Glass Items",
    4: "Other & Unlabeled Litter"
}

# Print ANNOTATION counts
print("\n📊 ANNOTATIONS PER CLASS:")
print("-"*60)
total_annotations = sum(annotation_counts.values())
for class_id in sorted(annotation_counts.keys()):
    count = annotation_counts[class_id]
    percentage = (count / total_annotations) * 100
    print(f"  Class {class_id}: {class_names_5[class_id]:<25} {count:6d} annotations ({percentage:5.1f}%)")

print("-"*60)
print(f"  TOTAL ANNOTATIONS: {total_annotations}")
print("="*60)

# Print IMAGE counts
print("\n📸 IMAGES PER CLASS:")
print("-"*60)
total_images = len(total_images_processed)
for class_id in sorted(image_counts.keys()):
    count = image_counts[class_id]
    percentage = (count / total_images) * 100
    print(f"  Class {class_id}: {class_names_5[class_id]:<25} {count:5d} images ({percentage:.1f}%)")

print("-"*60)
print(f"  TOTAL UNIQUE IMAGES: {total_images}")
print("="*60)

# Check 300-annotation threshold
print("\n🎯 TRAINING READINESS CHECK (target: 300+ annotations per class):")
print("-"*60)
all_good = True
for class_id in range(5):
    count = annotation_counts.get(class_id, 0)
    if count >= 300:
        status = "✅ READY"
    else:
        status = f"❌ NEEDS MORE (short by {300-count} annotations)"
        all_good = False
    print(f"  Class {class_id}: {class_names_5[class_id]:<25} {count:5d} annotations - {status}")

# Summary by material type

print("\n📋 MATERIAL BREAKDOWN:")
print("-"*60)
material_totals = {
    "Plastic (Class 0)": annotation_counts.get(0, 0),  # Removed sum()
    "Metal (Class 1)": annotation_counts.get(1, 0),    # Removed sum()
    "Paper (Class 2)": annotation_counts.get(2, 0),    # Removed sum()
    "Glass (Class 3)": annotation_counts.get(3, 0),    # Removed sum()
    "Other (Class 4)": annotation_counts.get(4, 0)     # Removed sum()
}

for material, count in material_totals.items():
    percentage = (count / total_annotations) * 100
    print(f"  {material:20}: {count:5d} annotations ({percentage:.1f}%)")

print("="*60)


TACO GROUPED INTO 5 CLASSES - CORRECTED MAPPING

📊 ANNOTATIONS PER CLASS:
------------------------------------------------------------
  Class 0: Plastic Items               2915 annotations ( 60.9%)
  Class 1: Metal Items                  203 annotations (  4.2%)
  Class 2: Paper & Cardboard           1288 annotations ( 26.9%)
  Class 3: Glass Items                  103 annotations (  2.2%)
  Class 4: Other & Unlabeled Litter     275 annotations (  5.7%)
------------------------------------------------------------
  TOTAL ANNOTATIONS: 4784

📸 IMAGES PER CLASS:
------------------------------------------------------------
  Class 0: Plastic Items              1038 images (69.2%)
  Class 1: Metal Items                 171 images (11.4%)
  Class 2: Paper & Cardboard           716 images (47.7%)
  Class 3: Glass Items                  86 images (5.7%)
  Class 4: Other & Unlabeled Litter    216 images (14.4%)
------------------------------------------------------------
  TOTAL UNIQUE IMAGE

**STEP 6 — move trans bin to class 6**

In [14]:


rf_labels = glob(f"{roboflow_path}/**/labels/*.txt", recursive=True)

for file in rf_labels:
    lines = []
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 0:
                parts[0] = "5"  # Move to class 5(trash bin)
                lines.append(" ".join(parts))

    if lines:  # Only write if there are lines
        with open(file, "w") as f:
            f.write("\n".join(lines))

print(f"Roboflow labels shifted to class 5 (trash bin). Processed {len(rf_labels)} files.")

Roboflow labels shifted to class 5 (trash bin). Processed 445 files.


**Create a combined Dataset**

In [17]:
# Remove old combined folder
shutil.rmtree("/content/combined", ignore_errors=True)

# Create combined structure
for split in ["train", "val", "test"]:
    os.makedirs(f"/content/combined/images/{split}", exist_ok=True)
    os.makedirs(f"/content/combined/labels/{split}", exist_ok=True)

# Copy TACO dataset (classes 0-4 for your 5-class scheme)
print("Copying TACO dataset (classes 0-4)...")
taco_dirs = {
    'train': '/content/taco_split/images/train',
    'val': '/content/taco_split/images/val',
    'test': '/content/taco_split/images/test'
}

taco_counts = {}
for split, path in taco_dirs.items():
    if os.path.exists(path):
        # Copy images
        !cp -r {path}/* /content/combined/images/{split}/ 2>/dev/null

        # Copy labels
        label_path = path.replace('images', 'labels')
        if os.path.exists(label_path):
            !cp -r {label_path}/* /content/combined/labels/{split}/ 2>/dev/null

        # Count copied items
        img_count = len(glob(f"/content/combined/images/{split}/*.jpg"))
        taco_counts[split] = img_count
        print(f"  Copied {img_count} TACO images to {split} (classes 0-4)")

# Copy Roboflow dataset (class 5 - trash bin)
print("\nCopying Roboflow dataset (class 5 - trash bin)...")

if os.path.exists(roboflow_path):
    roboflow_counts = {}
    for split in ['train', 'valid', 'test']:
        target_split = 'val' if split == 'valid' else split

        # Count images BEFORE copying (to calculate how many we add)
        before_count = len(glob(f"/content/combined/images/{target_split}/*.jpg"))

        # Copy images
        img_source = f"{roboflow_path}/{split}/images"
        if os.path.exists(img_source):
            !cp -r {img_source}/* /content/combined/images/{target_split}/ 2>/dev/null

        # Copy labels
        label_source = f"{roboflow_path}/{split}/labels"
        if os.path.exists(label_source):
            !cp -r {label_source}/* /content/combined/labels/{target_split}/ 2>/dev/null

        # Count AFTER copying and calculate the difference
        after_count = len(glob(f"/content/combined/images/{target_split}/*.jpg"))
        copied_count = after_count - before_count
        roboflow_counts[target_split] = copied_count
        print(f"  Copied {copied_count} Roboflow images to {target_split} (class 5)")

        # Verify it matches expected
        expected = len(glob(f"{img_source}/*.jpg")) if os.path.exists(img_source) else 0
        if copied_count != expected:
            print(f"    ⚠️  Expected {expected} but copied {copied_count} - possible duplicates?")
else:
    print(f"  Warning: Roboflow path {roboflow_path} does not exist!")
    print("  Please update the roboflow_path variable with the correct path.")

# Final verification - CORRECTED for 6 classes (0-5)
print("\n" + "="*60)
print("FINAL COMBINED DATASET COUNTS (6 total classes: 0-5)")
print("="*60)

# Define your 6 classes (5 TACO classes + 1 trash bin)
class_names_6 = {
    0: "Plastic Items",
    1: "Metal Items",
    2: "Paper & Cardboard",
    3: "Glass Items",
    4: "Other & Unlabeled Litter",
    5: "Trash Bin (Roboflow)"
}

total_images = 0
total_labels = 0

for split in ["train", "val", "test"]:
    images = glob(f"/content/combined/images/{split}/*.jpg")
    labels = glob(f"/content/combined/labels/{split}/*.txt")
    print(f"\n{split.upper():5} - Images: {len(images):6d}, Labels: {len(labels):6d}")
    total_images += len(images)
    total_labels += len(labels)

    # Check for missing labels
    missing = 0
    for img in images:
        label = img.replace('/images/', '/labels/').replace('.jpg', '.txt')
        if not os.path.exists(label):
            missing += 1
    if missing > 0:
        print(f"         ⚠️  Warning: {missing} images missing labels in {split}")
    else:
        print(f"         ✅ All images have corresponding labels")

    # Check class distribution in this split (only classes 0-5)
    class_counts = {i:0 for i in range(6)}
    for label_file in labels:
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    parts = line.strip().split()
                    if len(parts) > 0:
                        class_id = int(parts[0])
                        if class_id in class_counts:
                            class_counts[class_id] += 1
                        else:
                            print(f"         ⚠️  Warning: Found class {class_id} outside range 0-5 in {label_file}")

    print(f"         Class distribution in {split}:")
    for i in range(6):
        if class_counts[i] > 0:
            print(f"           Class {i} ({class_names_6[i]}): {class_counts[i]:6d} annotations")

print("="*60)
print(f"TOTAL - Images: {total_images}, Labels: {total_labels}")
print("="*60)

# Summary by dataset source
print("\n" + "="*60)
print("DATASET SOURCE SUMMARY")
print("="*60)
print(f"TACO dataset (classes 0-4):")
for split, count in taco_counts.items():
    print(f"  {split}: {count} images")
print(f"\nRoboflow dataset (class 5):")
for split, count in roboflow_counts.items():
    print(f"  {split}: {count} images")

# Calculate and verify totals
total_taco = sum(taco_counts.values())
total_robo = sum(roboflow_counts.values())
print(f"\nVERIFICATION:")
print(f"  Total TACO images: {total_taco}")
print(f"  Total Roboflow images: {total_robo}")
print(f"  Combined total: {total_taco + total_robo}")
print(f"  Actual total in folder: {total_images}")
if total_taco + total_robo == total_images:
    print(f"  ✅ Counts match!")
else:
    print(f"  ❌ Counts don't match!")
print("="*60)

Copying TACO dataset (classes 0-4)...
  Copied 1200 TACO images to train (classes 0-4)
  Copied 150 TACO images to val (classes 0-4)
  Copied 150 TACO images to test (classes 0-4)

Copying Roboflow dataset (class 5 - trash bin)...
  Copied 314 Roboflow images to train (class 5)
  Copied 105 Roboflow images to val (class 5)
  Copied 26 Roboflow images to test (class 5)

FINAL COMBINED DATASET COUNTS (6 total classes: 0-5)

TRAIN - Images:   1514, Labels:   1514
         ✅ All images have corresponding labels
         Class distribution in train:
           Class 0 (Plastic Items):   2220 annotations
           Class 1 (Metal Items):    156 annotations
           Class 2 (Paper & Cardboard):   1013 annotations
           Class 3 (Glass Items):     84 annotations
           Class 4 (Other & Unlabeled Litter):    218 annotations
           Class 5 (Trash Bin (Roboflow)):    398 annotations

VAL   - Images:    255, Labels:    255
         ✅ All images have corresponding labels
         Clas

count the images in each category -> train,val and test

**step7: Create YOLO data.yaml**

In [18]:

# Define our 6 classes (TACO 5 classes + Trash Bin)
final_names = [
    # TACO grouped classes (5 classes) - classes 0-4
    "plastic_items",           # class 0
    "metal_items",             # class 1
    "paper_cardboard",         # class 2
    "glass_items",             # class 3
    "other_litter",            # class 4

    # Roboflow class (1 class) - class 5
    "trash_bin"                # class 5
]

# Create combined.yaml
with open("combined.yaml", "w") as f:
    f.write("# TACO (5 classes: 0-4) + Trash Bin (1 class: 5) dataset\n\n")
    f.write("train: /content/combined/images/train\n")
    f.write("val: /content/combined/images/val\n")
    f.write("test: /content/combined/images/test\n\n")
    f.write(f"nc: {len(final_names)}  # 6 classes total (0-5)\n")
    f.write("names:\n")
    for name in final_names:
        f.write(f"  - {name}\n")

print("✅ combined.yaml created with 6 classes (0-5):")
for i, name in enumerate(final_names):
    print(f"   Class {i}: {name}")

# Verify the paths exist
print("\nVerifying paths:")
total_images = 0
for split in ["train", "val", "test"]:
    path = f"/content/combined/images/{split}"
    if os.path.exists(path):
        count = len(glob(f"{path}/*.jpg"))
        total_images += count
        print(f"  {split}: {path} - {count:4d} images ✅")
    else:
        print(f"  {split}: {path} - NOT FOUND ❌")

print(f"\n📊 TOTAL IMAGES IN COMBINED DATASET: {total_images}")
print("="*50)

# Quick class distribution check
print("\n📋 Expected class mapping:")
print("-"*40)
class_desc = {
    0: "Plastic Items (bottles, bags, wrappers, film, etc.)",
    1: "Metal Items (cans, foil, caps, etc.)",
    2: "Paper & Cardboard (boxes, cups, cartons, etc.)",
    3: "Glass Items (bottles, jars, broken glass)",
    4: "Other Litter (cigarettes, organic waste, electronics, textiles)",
    5: "Trash Bin (from Roboflow dataset)"
}

for i in range(6):
    print(f"  Class {i}: {final_names[i]}")
    print(f"        {class_desc[i]}")
print("="*50)


✅ combined.yaml created with 6 classes (0-5):
   Class 0: plastic_items
   Class 1: metal_items
   Class 2: paper_cardboard
   Class 3: glass_items
   Class 4: other_litter
   Class 5: trash_bin

Verifying paths:


AttributeError: 'function' object has no attribute 'glob'

# **8: — Train YOLOv11**

In [ ]:

# Path to your combined YAML
data_yaml = "combined.yaml"

# Use YOLO11n pretrained on COCO as base
model = YOLO("yolo11n.pt")

# Training parameters - saving to default path /content/runs/detect/train/
model.train(
    data=data_yaml,        # 6-class dataset (0-5)
    epochs=100,            # max epochs
    imgsz=640,             # image size
    batch=16,              # adjust to GPU memory
    device=device,         # device is already defined in previous cells
    patience=10,           # early stopping if val mAP not improving for 10 epochs
    save=True,             # saves best.pt automatically
    pretrained=True,       # start from COCO weights
    optimizer='auto',      # YOLO chooses best optimizer
    lr0=0.01,              # initial learning rate
    augment=True,          # enables basic augmentations

    # Augmentation parameters
    mosaic=1.0,            # mosaic augmentation (0-1 probability)
    mixup=0.1,             # mixup augmentation (0-1 probability)
    flipud=0.0,            # vertical flip - 0.0 as trash isn't usually upside down
    fliplr=0.5,            # horizontal flip - useful for symmetry
    hsv_h=0.015,           # hue jitter
    hsv_s=0.7,             # saturation jitter
    hsv_v=0.4,             # value (brightness) jitter
    degrees=0.0,           # rotation (0 for no rotation - keeps annotations valid)
    translate=0.1,         # translation
    scale=0.5,             # scaling
    shear=0.0,             # shear (keep 0 to avoid distortion)

    # Training settings
    deterministic=True,    # reproducible results
    save_period=5,         # save weights every 5 epochs
    plots=True,            # generate loss/metrics plots automatically
    workers=8,             # number of dataloader workers
    seed=42,               # for reproducibility
    verbose=True           # print detailed output
    # project and name are removed - will use default: runs/detect/train/
)


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=combined.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False,

## **STEP 6 — Download trained model**

In [ ]:

files.download("/content/runs/detect/train4/weights/best.pt")


FileNotFoundError: Cannot find file: /content/runs/detect/train4/weights/best.pt